In [ ]:
import asyncio
from typing import Awaitable, Sequence, Literal, Tuple
import freespeech.client as FreeSpeech
from freespeech.types import Event, Language
from dataclasses import dataclass


ORIGIN_URL = "https://www.youtube.com/watch?v=DEqXNfs_HhY"
ORIGIN_LANG = 'en-US'
TARGET_LANG = 'ru-RU'
METHOD = "Subtitles"

HELLO_WORLD_EN = Event(time_ms=0, chunks=["Hello world"])


SpeechToText = Literal["C3PO", "R2D2", "BB8"]
DocumentFormat = Literal["Google", "Notion", "SRT"]


@dataclass
class Source:
    method: Literal[SpeechToText, "Subtitles", "Translate"]
    url: str


@dataclass(frozen=True)
class Audio:
    duration_ms: int
    encoding: AudioEncoding
    sample_rate_hz: int
    num_channels: int


@dataclass(frozen=True)
class Video:
    duration_ms: int
    encoding: VideoEncoding
    # TODO (astaff): add fps, HxW, etc


@dataclass(frozen=True)
class Settings:
    original_audio_level: int = 2
    gaps: Literal["Crop", "Blank", "Fill"] = "Blank"


@dataclass
class Transcript:
    title: str | None
    lang: Language
    events: Sequence[Event]
    origin: Source | None
    audio: Tuple[str, Audio] | None
    video: Tuple[str, Video] | None
    settings: Settings


class FreeSpeech:
    def __init__(self, key: str):
        self._client = aiohttp.Client("https://api.url/endpoint", headers={f"Auth-Token: {key}"})


    async def get_transcript(
        *,
        url: str,
        lang: Language,
        method: Literal[SpeechToText, "Subtitles"]
    ) -> Tuple[Response, Awaitable[Transcript]]:
        """Given source URL, BCP-47 language tag, and one of machine transcrtiption methods, or "Subtitles"
        return Transcript.

        Args:
            url: YouTube (future: FB, IG, TG, TT, HTTP)
            lang: languge tag, i.e. "en-US", "uk-UA" (BCP-47, RFC5646)
            method: one of the machine backends (i.e. "R2D2" or "C3PO") or "Subtitles".

        Returns:
            Transcript.
        """
        pass


    async def load(*, url: str) -> Transcript:
        """Load and parse transcript from one of the supported backends: Notion, Google Doc, SRT??"""
        pass


    async def save(*, transcript: Transcript, format: DocumentFormat) -> str:
        """Save transcript into Notion, Google, SRT (specified in `format`)."""
        pass


    async def translate(*, transcript: Transcript, lang: Language) -> Tuple[Response, Awaitable[Transcript]]:
        """Given transcript, translate it to language specified as BCP-47 tag
        """
        pass


    async def synth(*, transcript: Transcript) -> Tuple[Response, Awaitable[str]]:
        """Synthesize and do voiceover according to transcript and transcript.settings.
        
        Returns:
            URL for synthesized media.
        """
        pass


    async def jobs() -> Sequence[Awaitable[Transcript]]:
        pass


fs = FreeSpeech(key="API-KEY")

response, task = await fs.generate(source="https://www.youtube.com/watch?v=DEqXNfs_HhY", lang=ORIGIN_LANG, method=METHOD)

if response.error:
    raise ClientResponseError(f"{response.error.message}")
transcript = await task


transcript = await fs.load(source="https://googledocs")
transcript = await fs.load(source="https://notion.so")

# How about SRT in Google? :-)
gdoc_url = await fs.save(transcript, format="Google")
notion_url = await fs.save(transcript, format="Notion")


respoinse, task = fs.translate(
    transcript=transcript,
    lang=TARGET_LANG
)
target_transcript = await task

response, task = await fs.synth(transcript=await fs.load("https://googledocs"))
response, task = await fs.synth(transcript=await fs.load("https://notion.so"))
response, task = await fs.synth(transcript=target_transcript)


results = await asyncio.gather(*fs.jobs())